# 知識點
## Two Stage
- R-CNN
- Fast R-CNN
- Faster R-CNN

### R-CNN: Regions with CNN features
首先會由 Selective search 提出 Region proposal，將它們 resize 到相同大小後再經由 CNN 提取特徵，最後使用 SVM 進行分類，分類結束後再用 Regressor 修正 BBox。
<img src="https://miro.medium.com/proxy/1*GPBlt7i3IY1MWSTwSHodHA.png" width=500>
其缺點就是速度慢，原因有二：
- 經由 Selective Search 提出的 Region Proposal 都要獨自經過CNN 做特徵提取，運算速度相當緩慢。
- 經過 CNN 得到 feature map，再用這些 feature 當成 SVM 的 input 當成訓練資料，因此並不是一個 end-to-end 模型 (SVM 的 LOSS 並不會改動到 feature map 的數值)。

### Fast R-CNN
使用 Softmax 取代 SVM，並且同時做 BBox regression，因此訓練與推論時間都大幅提昇。

實際上 Fast R-CNN 解決了以下兩個問題：
- 原本每一個 Region Proposal 都要經過 CNN 提取特徵現在只需要將整張圖送入，速度上有很大的優勢。
- 運用 Softmax 取代 SVM，並同時加入了 BBOX regression 到模型中與分類一起收斂。

<img src="https://miro.medium.com/proxy/1*mBZ6At0eap6Mp6ssbUD3rA.png" width=500>
其中 ROI 指的就是 Region proposal，而 ROI Pooling 是一種能將不同 Region proposals (候選框)，Pooling 成一樣大小的一個方法。

Fast R-CNN 還剩以下問題：
- 需要透過 Selective Search 提出 Region Proposal，沒辦法統整成 End-to-End model 訓練。

### Faster R-CNN
改用 RPN 提出 Region Proposal，其餘結構與 Fast R-CNN 相同，因此在速度上完勝其他 Two-Stage 模型 (包含 [SPPNet](https://zhuanlan.zhihu.com/p/24774302))。

實際上 Fast R-CNN 解決了以下問題：
- 不用 Selective Search，而改用 (Region Proposal Network) 來提取 proposals ，達到真正的 End-to-End。

<img src="https://pic3.zhimg.com/80/v2-c0172be282021a1029f7b72b51079ffe_720w.jpg" width=250>

Faster R-CNN 還剩以下問題：
- 還是需要先提出 Region Proposal，整體而言速度受到一定的限制。

參考 [Object Detection : R-CNN, Fast-RCNN, Faster RCNN](https://medium.com/@ccshenyltw/object-detection-r-cnn-fast-rcnn-faster-rcnn-mask-rcnn-retinanet-to-be-continued-71b67640445)

## One Stage

為了解決 Two-Stage 需要提出 proposals 導致速度變慢的問題，以追求速度為主要目的的 One-stage 模型開始出現。

- YOLO系列
- SSD
- RetinaNet

> 核心觀念-不要先浪費時間提出 Region Proposal，而是以 Default Anchor Box 取代。

### YOLO: You Only Look Once
下圖為 YOLO 與 RCNN 系列的比較：
<img src="https://img-blog.csdnimg.cn/20181122204900397.jpg?x-oss-process=image/watermark,type_ZmFuZ3poZW5naGVpdGk,shadow_10,text_aHR0cHM6Ly9ibG9nLmNzZG4ubmV0L0dlbnRsZW1hbl9RaW4=,size_16,color_FFFFFF,t_70" width=500>

#### YOLOv1
YOLOv1 架構如下，其中圖片輸入後會直接以 CNN 提取特徵，接著展平通過 FC 層，再 resize 回 7×7×30，最後的 loss 為 softmax分類 ＋ BBox回歸 ＋ confidence：
<img src="https://gitthhub.github.io/img/yolov1_net.png" width=500>

YOLOv1 的創新想法為：運用最後一層特徵圖中每一個像素中心點( Default Anchor Box )直接產生 Region Poposal，可以想像成直接以像素中心點來劃框框，而原文是每一個像素會有兩個預設 Bounding Box，如此一來便不用花費時間提出 Region Proposal。

<img src="https://shenheng.xyz/img/1_JniWRt-ceWLNlkOULjhdpg.png" width=400>

運用最後一層的 Feature Map 直接畫候選框，最後一層有 7×7=49 個像素點，YOLOv1 在每一像素點上設計兩個候選框，因此有 98 個候選框，每個框需要預測三個 Output，分別是：
- 屬於哪一類 (Classification)
- 校正BBox (Regression)
- Confidence (有物件的信心程度)

YOLOv1 的問題有二：
- 只透過最後一層預測，而由於最後一層的 Feature Map 尺寸只有7×7，已經喪失許多空間訊息，因此對小物件不敏感。
- FC層造成空間訊息損失。

#### YOLOv2、v3
發表於 SSD 之後，針對 V1 的問題做改善，以後再詳述。

### SSD: Single Shot MultiBox Detector

SSD 運用多尺度 Feature Maps、不使用 FC 層，並且每一個像素上放置更多 default anchor BOX，全部一共有 8732 個( 相較於 YoloV1 只有 98個 )，因此對小物件更敏感。
<img src="https://i.iter01.com/images/12ee06051c1e8c8d777eb0afad64c903de216be8723cf95bb90ad133484e431b.png" width=500>

SSD 的問題有二：
- 正負樣本不平均，太多背景，使用 [Online hard example mining ( OHEM )](https://www.itread01.com/content/1543549147.html) 使易分類樣品消失。
- 運用淺層 Feature Map 偵測小物件，但特徵訊息不夠豐富，對小物件敏感度仍比不上 Faster R-CNN。

### RetinaNet

RetinaNet 的主架構與 SSD 很像，重點在於加入了 Feature Pyramid Network (FPN) 的結構，把淺層的 Feature Map 與深層的 Feature Map 疊加後做預測，確保淺層語義訊息也能夠豐富，因此對小物件更敏感。
<img src="https://paperswithcode.com/media/methods/Screen_Shot_2020-06-07_at_4.22.37_PM.png" width=600>

另一個重點就是設計 Focal Loss，用來解決背景預測框太多的狀況，並同時處理類別樣本不均的狀況:
- 在 RetinaNet 中，Default Anchor Box 有 10 萬個左右，其中真正有涵蓋物件的候選框少之又少(可能只有幾百個)，如果不經過任何處理，參數的更新主要都被負樣品影響 (SSD中用OHEM解決)。
- 除了正負樣品( 候選框 )不均外，類別不均也是常見的問題，如包含狗的照片很多，包含貓的照片很少。
<img src="https://paperswithcode.com/media/methods/Screen_Shot_2020-06-07_at_4.45.06_PM_leJm2yh.png" width=300>

Focal Loss 為 Cross Entropy 的變形，其中 $p_t$ 是我們預測的機率值，將每個 $p_t$ 賦予一個 $(1-p_t)^\gamma$ 的權重，其中 $\gamma$ 為超參數，當預測機率值越高 (易分類樣品)，其所獲得的權重會越低，藉此抑制易分類的背景框，同時藉由 $\alpha$ 賦予不同類別不同的權重 ($\alpha$ 常見為一個List，ex. [1.0, 0.1])。

RetinaNet 的問題如下：
- Focal Loss 中 Hyperparameters 的調整對結果影響大。

##  Home work
- 了解 Two Stage 為何通常擁有較高的精度。
- 了解 One Stage 的 Default Anchor Box 概念。
- 了解 SSD 相較於 YoloV1 的優勢。
- 了解 Focal Loss 設計的核心理念，如何處理易分類樣品與類別不均。
> 參考 [Focal Loss(RetinaNet) 與 OHEM](https://www.itread01.com/content/1543549147.html)